# Tutorial 2.3: One-Body Potential Energy Surface

In this section, we construct, from scratch, a 1-body MB-nrg-style PEF for CH3-NH2.

## 2.3.0 Definitions and Imports

In [ ]:
%run Tutorial_2_Imports.ipynb

In [ ]:
%run Tutorial_2_Paths.ipynb

## 2.3.1 Normal Modes Configuration Generation

Now, we will generate the first batch of configurations for our training set. This batch will come from randomly sampling the vibrational normal modes of methylamine.

We can read in the minimized structure and modes we calculated earlier:

In [ ]:
definition_CH3NH2 = read_definition(paths.definitions.systems.CH3NH2)
minimized_structure_CH3NH2, minimized_energy_CH3NH2 = read_system_and_energy(definition_CH3NH2, paths.structures.minimized_CH3NH2)
vibrational_modes_CH3NH2 = read_vibrational_modes(paths.structures.vibrational_modes_CH3NH2)

And then generate 1000 configurations by randomly sampling the normal modes:

In [ ]:
configuration_generator = NormalModesConfigurationGenerator(
        minimized_structure_CH3NH2,
        vibrational_modes_CH3NH2,
)

configurations_vibrational_modes_CH3NH2 = list(configuration_generator.generate_configurations(num_configs=1000, seed=54321))
write_systems(paths.PIP_CH3NH2.configs.vibrational_modes, configurations_vibrational_modes_CH3NH2)

Take a peek at the configurations:

In [ ]:
render_overlayed_systems(
    configurations_vibrational_modes_CH3NH2,
    centerer=lambda system: system.atoms[0].point,
    aligner=lambda system: (system.atoms[4].point, system.atoms[3].point),
    alpha=0.05,
    num_to_show=25
)

## 2.3.2 Parallel-Bias Metadynamics Configuration Generation

Let's generate the second set of configurations for our training set, those sampled from metadynamics simulations.

We need some force-field params to do the metadynamics with, since we want to generate very diverse configurations, we choose very weak force constants:

In [ ]:
bond_params, angle_params, nonbonded_params = get_pbmetad_parameters(
        minimized_structure_CH3NH2
)

In [ ]:
bond_params

In [ ]:
angle_params

In [ ]:
nonbonded_params

And we can use the charges we calculated earlier, but we will scale them by 1/3, to allow the simulation to reach more distorted configurations:

In [ ]:
charges = read_parameters(paths.Vphys.charges)
charges = {symmetry: charges[symmetry]*1/3 for symmetry in charges}

In [ ]:
charges

Then, we run the metadynamics simulation:

In [ ]:
perform_pbmetad_simulation(
        lammps_executable_path,
        [minimized_structure_CH3NH2],
        num_configs=500000,
        sample_interval=10,
        temperature=700,
        seed=12345,
        configurations_path=paths.PIP_CH3NH2.configs.pbmetad_pool,
        bond_params=bond_params,
        angle_params=angle_params,
        nonbonded_params=nonbonded_params,
        charges=charges,
        pbmetad_workdir=paths.scratch.pbmetad_CH3NH2,
        restart_path=paths.restart.pbmetad_CH3NH2
)

We can take 200 configurations out to use as a test set later:

In [ ]:
split_pool(
        definition_CH3NH2,
        paths.PIP_CH3NH2.configs.pbmetad_pool,
        pool_size=500000,
        num_configs_to_split=200,
        unsplit_path=paths.PIP_CH3NH2.configs.pbmetad_train_pool,
        split_path=paths.PIP_CH3NH2.configs.pbmetad_test,
        strategy="INTERVAL",
)

Then, we can use farthest point sampling to choose the ~1000 configurations that give a good broad coverge of the space:

In [ ]:
pbmetad_configs_CH3NH2 = perform_fps(
        definition_CH3NH2,
        minimized_structure_CH3NH2,
        vibrational_modes_CH3NH2,
        paths.PIP_CH3NH2.configs.pbmetad_train_pool,
        num_pool_configs=500000-200,
        num_phase1_input_configs=100000,
        approx_configs_to_select=1000,
        fps_workdir=paths.scratch.fps_CH3NH2,
        restart_path=paths.restart.fps_CH3NH2,
        num_threads=16
)

Clearly, the configurations from metadynamics + FPS give a very broad coverge of the configuration space:

In [ ]:
render_overlayed_systems(
    pbmetad_configs_CH3NH2,
    centerer=lambda system: system.atoms[0].point,
    aligner=lambda system: (system.atoms[4].point, system.atoms[3].point),
    alpha=0.05,
    num_to_show=25
)

In [ ]:
write_systems(paths.PIP_CH3NH2.configs.pbmetad_train, pbmetad_configs_CH3NH2)

## 2.3.3 Energy Calculation

In [ ]:
calculator = Psi4Calculator(
        "MP2",
        "def2-TZVP",
        log_directory=paths.logs,
        scratch_directory=paths.scratch.psi4
)

In [ ]:
energies_vibrational_modes_CH3NH2 = calculate_nmer_energies(
        configurations_vibrational_modes_CH3NH2,
        calculator,
        paths.restart.energies_vibrational_mode_configs_CH3NH2,
        num_threads=8,
        mem_mb=8000,
        error_on_any_failure=False,
        retrieve_only=True
)
write_systems_and_nmer_energies(paths.PIP_CH3NH2.configs.vibrational_modes_energies, (configurations_vibrational_modes_CH3NH2, energies_vibrational_modes_CH3NH2))

In [ ]:
energies_pbmetad_CH3NH2 = calculate_nmer_energies(
        pbmetad_configs_CH3NH2,
        calculator,
        paths.restart.energies_pbmetad_configs_CH3NH2,
        num_threads=8,
        mem_mb=8000,
        error_on_any_failure=False,
        retrieve_only=True
)
write_systems_and_nmer_energies(paths.PIP_CH3NH2.configs.pbmetad_train_energies, (pbmetad_configs_CH3NH2, energies_pbmetad_CH3NH2))

In [ ]:
configurations_vibrational_modes_CH3NH2, energies_vibrational_modes_CH3NH2 = discard_failed_configurations(
        configurations_vibrational_modes_CH3NH2,
        energies_vibrational_modes_CH3NH2
)

pbmetad_configs_CH3NH2, energies_pbmetad_CH3NH2 = discard_failed_configurations(
        pbmetad_configs_CH3NH2,
        energies_pbmetad_CH3NH2
)

In [ ]:
energies_1b_vibrational_modes_CH3NH2 = calculate_many_body_energies(
    configurations_vibrational_modes_CH3NH2,
    energies_vibrational_modes_CH3NH2,
    minimized_monomer_energies=[minimized_energy_CH3NH2]
)

energies_1b_pbmetad_CH3NH2 = calculate_many_body_energies(
    pbmetad_configs_CH3NH2,
    energies_pbmetad_CH3NH2,
    minimized_monomer_energies=[minimized_energy_CH3NH2]
)

In [ ]:
write_training_set_file(
        paths.PIP_CH3NH2.training_set,
        (
                [*configurations_vibrational_modes_CH3NH2, *pbmetad_configs_CH3NH2],
                [*energies_1b_vibrational_modes_CH3NH2, *energies_1b_pbmetad_CH3NH2],
                [*energies_1b_vibrational_modes_CH3NH2, *energies_1b_pbmetad_CH3NH2],
        )
)
print(f"Total configurations for training set: {len(configurations_vibrational_modes_CH3NH2) + len(pbmetad_configs_CH3NH2)}")

In [ ]:
min(energies_1b_vibrational_modes_CH3NH2), max(energies_1b_vibrational_modes_CH3NH2)

In [ ]:
min(energies_1b_pbmetad_CH3NH2), max(energies_1b_pbmetad_CH3NH2)

## 2.3.4 Polynomial Generation

In [ ]:
pip_1b_max_order = 3

In [ ]:
generate_polynomial(
        definition_CH3NH2,
        max_degree=pip_1b_max_order,
        polynomial_info_path=paths.PIP_CH3NH2.PIP.info,
        polynomial_dir=paths.PIP_CH3NH2.PIP.polynomial,
        filters=[]
)

## 2.3.5 Polynomial Parametrization

In [ ]:
polarizabilities = read_parameters(paths.Vphys.polarizabilities)
polarizability_factors = read_parameters(paths.Vphys.polarizability_factors)
charges = read_parameters(paths.Vphys.charges)
C6 = read_parameters(paths.Vphys.C6)
d6 = read_parameters(paths.Vphys.d6)

In [ ]:
print("polarizabilities:", polarizabilities)
print("polarizability_factors:", polarizability_factors)
print("charges:", charges)
print("C6:", C6)
print("d6:", d6)

In [ ]:
generate_pip_parametizer(
        definition_CH3NH2,
        max_degree=pip_1b_max_order,
        polynomial_info_path=paths.PIP_CH3NH2.PIP.info,
        polynomial_dir=paths.PIP_CH3NH2.PIP.polynomial,
        parametizer_dir=paths.PIP_CH3NH2.PIP.parameterizer,
        workdir=paths.scratch.PIP_CH3NH2,
        symmetry_to_charge=charges,
        symmetry_to_polarizability=polarizabilities,
        symmetry_to_polarizability_factor=polarizability_factors,
        symmetries_to_C6=C6,
        symmetries_to_d6=d6,
        monomer_names=["ch3nh2"],
        is_mbpol=[False]
)

In [ ]:
parametrize_pip(
        definition_CH3NH2,
        parametizer_dir=paths.PIP_CH3NH2.PIP.parameterizer,
        parametrization_dir=paths.PIP_CH3NH2.PIP.parametrization,
        training_set_path=paths.PIP_CH3NH2.training_set,
        workdir=paths.scratch.fitting_CH3NH2,
        monomer_names=["ch3nh2"],
        is_mbpol=[False],
        num_fits_to_run=1,
        num_threads=16,
        DE=10,
        alpha=0.0005
)

## 2.3.6 Implementation in MBX

In [ ]:
add_PIP_to_MBX(
        paths.MBX,
        definition_CH3NH2,
        ["ch3nh2"],
        paths.PIP_CH3NH2.PIP.parameterizer,
        paths.PIP_CH3NH2.PIP.parametrization,
        max_degree=pip_1b_max_order,
        workdir=paths.scratch.fitting_CH3NH2,
)

In [ ]:
compile_mbx(
        paths.MBX,
        configure_arguments=[
                "CXX=icpc",
                "CC=icc",
                "--disable-optimization"
        ],
        configure=True,
        clean=False,
        num_threads=4
)

## 2.3.7 Correlation & Validation

In [ ]:
predicted_energies_vibrational_modes_CH3NH2 = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2,
        configurations_vibrational_modes_CH3NH2,
        monomer_names=["ch3nh2"],
        scratch_dir=paths.scratch.fitting_CH3NH2
)

In [ ]:
predicted_energies_pbmetad_CH3NH2 = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2,
        pbmetad_configs_CH3NH2,
        monomer_names=["ch3nh2"],
        scratch_dir=paths.scratch.fitting_CH3NH2
)

In [ ]:
mbx_opt_CH3NH2_energy = calculate_MBX_energy(
        paths.MBX,
        definition_CH3NH2,
        minimized_structure_CH3NH2,
        monomer_names=["ch3nh2"],
        scratch_dir=paths.scratch.fitting_CH3NH2
)

In [ ]:
predicted_energies_1b_vibrational_modes_CH3NH2 = calculate_many_body_energies(
    configurations_vibrational_modes_CH3NH2,
    predicted_energies_vibrational_modes_CH3NH2,
    minimized_monomer_energies=[mbx_opt_CH3NH2_energy]
)

predicted_energies_1b_pbmetad_CH3NH2 = calculate_many_body_energies(
    pbmetad_configs_CH3NH2,
    predicted_energies_pbmetad_CH3NH2,
    minimized_monomer_energies=[mbx_opt_CH3NH2_energy]
)

In [ ]:
plot_correlation(
    configurations_vibrational_modes_CH3NH2,
    energies_1b_vibrational_modes_CH3NH2,
    "MP2",
    {"MBX": predicted_energies_1b_vibrational_modes_CH3NH2},
    error_width=1.0
)

In [ ]:
plot_correlation(
    configurations_vibrational_modes_CH3NH2,
    energies_1b_vibrational_modes_CH3NH2,
    "MP2",
    {"MBX": predicted_energies_1b_vibrational_modes_CH3NH2},
    minE=-1,
    maxE=10,
    error_width=1.0
)

In [ ]:
plot_correlation(
    pbmetad_configs_CH3NH2,
    energies_1b_pbmetad_CH3NH2,
    "MP2",
    {"MBX": predicted_energies_1b_pbmetad_CH3NH2},
    error_width=10.0
)